# <p style="background-color:#50727B;font-family:Segoe UI;color:#FFFFF;font-size:120%;text-align:center;border-radius:10px 10px;">Modelado y Generación Musical: Una Aproximación Mediante Redes Neuronales y Entradas MIDI</p>


<p style="font-family:Segoe UI;font-size:120%;color:#FFFFF;">Welcome to this Jupyter Notebook, the digital canvas where my two greatest passions converge: music and machine learning. At the heart of this exploration lies a fascination with the intricate dance between technology and melody, an adventure into how ML can transform MIDI notes into captivating symphonies. As we embark on this journey together, we delve into the algorithms and creativity that fuel the future of music generation. Music, my utmost passion, serves as both the inspiration and the destination for this academic endeavor. Join me as we uncover the melodies that lie waiting in the realm of bits and bytes, crafting harmonies guided by the science of artificial intelligence.</p> 

<div class="list-group" id="list-tab" role="tablist">
 <p style="font-family:newtimeroman;color:#97BACB#97BACB;font-size:120%;text-align:center;border-radius:40px 40px;">TABLE OF CONTENTS</p>

# <p style="background-color:#50727B;font-family:Segoe UI;color:#FFFFF;font-size:120%;text-align:center;border-radius:10px 10px;">Install Dependencies
</p>